Imports

In [32]:
import pandas as pd
import numpy as np 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

import os

Make sure we are in the right directory

In [33]:
# os.chdir('Documents/Deep-Learning-Chronicles/air_quality_prediction/')
# os.getcwd()

Load the data

In [34]:
df = pd.read_csv("smoke_occurence_nairobi.csv")
df.head()

,id,time_opening_windows,time_closing_windows,smoke_detected,time_sensing_smoke,duration,date,day,occassion,weather,type_of_smoke
0,1,1641,1717,NaN,1717.0,NaN,2025-10-03,Friday,nothing,cloudywithoutwind,stove
1,2,1252,1528,NaN,1728.0,NaN,2025-10-04,Saturday,NaN,cloudywithoutwind,stove
2,3,1452,1734,NaN,1734.0,NaN,2025-10-05,Sunday,NaN,windy,NaN
3,4,1035,1759,NaN,1759.0,NaN,2025-10-06,NaN,NaN,windy,stove
4,5,1400,1813,True,1813.0,NaN,2025-10-07,Tuesday,NaN,NaN,NaN


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    20 non-null     int64  
 1   time_opening_windows  20 non-null     int64  
 2   time_closing_windows  20 non-null     int64  
 3   smoke_detected        14 non-null     object 
 4   time_sensing_smoke    8 non-null      float64
 5   duration              12 non-null     float64
 6   date                  20 non-null     object 
 7   day                   19 non-null     object 
 8   occassion             12 non-null     object 
 9   weather               16 non-null     object 
 10  type_of_smoke         6 non-null      object 
dtypes: float64(2), int64(3), object(6)
memory usage: 1.8+ KB


Basic cleaning

In [36]:
df["smoke_detected"] = (
    df["smoke_detected"]
    .fillna(False)
    .astype(str)
    .str.lower()
    .map({"true": 1, "false": 0})
)

df["smoke_detected"].value_counts()

C:\Users\ianwr\AppData\Local\Temp\ipykernel_7396\3610187339.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .fillna(False)


smoke_detected
0    18
1     2
Name: count, dtype: int64

In [37]:
def time_to_minutes(t):
    if pd.isna(t) or t == "":
        return np.nan
    t = str(t).zfill(4)
    final = int(t[:2]) * 60 + int(t[2:])
    return final

df["window_start_min"] = df["time_opening_windows"].apply(time_to_minutes)
df["window_end_min"] = df["time_closing_windows"].apply(time_to_minutes)
df["window_length_min"] = df["window_end_min"] - df["window_start_min"]

df[["window_start_min", "window_end_min", "window_length_min"]].head()


,window_start_min,window_end_min,window_length_min
0,1001,1037,36
1,772,928,156
2,892,1054,162
3,635,1079,444
4,840,1093,253
